# Reward model JERX task

In [ ]:
#|default_exp jerx.reward.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
import os
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI, BadRequestError

from bellek.text.utils import fuzzy_match
from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# |export

DEFAULT_SYSTEM_PROMPT = """You are an expert Q&A system that is trusted around the world. You are given a question that requires multi-hop reasoning. Always answer the question using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
"""

USER_PROMPT = """The context information below is provided as a set of entity-relation-entity triplets from knowledge graph.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the question.
{question}
"""

class QuestionAnsweringResult(BaseModel):
    """Data model for answering the question."""

    answer: str = Field(description="The answer to the question in 2-4 words.")
    reasoning: str = Field(description="Multi-hop reasoning for the answer.")


def make_question_answer_func(model_name: str = "gpt-3.5-turbo", client: OpenAI = None):
    if client is None:
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_API_BASE"))
    client = instructor.from_openai(client)

    def func(context: str, question: str) -> QuestionAnsweringResult:
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": USER_PROMPT.format(context=context, question=question),
            },
        ]
        return client.chat.completions.create(
            model=model_name,
            response_model=QuestionAnsweringResult,
            messages=messages,
        )

    return func

In [ ]:
#|export

class RewardAssessment(QuestionAnsweringResult):
    reward: float = Field(description="The reward value for the answer.")

def make_reward_func(model_name: str = "gpt-3.5-turbo", answer_comparator=fuzzy_match):
    qa = make_question_answer_func(model_name)

    def reward(context: str, question: str, answers: list[str]) -> RewardAssessment:
        try:
            qa_result = qa(context, question)
        except BadRequestError as e:
            log.warning(f"Failed to assess generation: {e}")
            return RewardAssessment(answer="", reasoning=str(e), reward=0.0)
        correct = any(answer_comparator(qa_result.answer, answer) for answer in answers)
        reward = 1.0 if correct else 0.0
        return RewardAssessment(**qa_result.dict(), reward=reward)

    return reward

In [ ]:
triplets = [ "Dominica | first competed at | Olympic Games in 1996", "Dominica | has participated in | each Games since then", "Dominica | has not won | any medals at the Olympic Games" ]

context = "\n".join(triplets)
question = "When did the country where the River Quanery  is found first compete in Olympic games?"
answer = "1996"

qa = make_question_answer_func()
qa(context=context, question=question)

QuestionAnsweringResult(answer='1996', reasoning='The country where the River Quanery is found first competed in Olympic Games in 1996, as mentioned in the context.')

In [ ]:
# reward_func = make_reward_func("llama3-70b-8192")
# assert reward_func(context, question, [answer]).reward == 1

In [ ]:
reward_func = make_reward_func("gpt-4-turbo")
assert reward_func(context, question, [answer]).reward == 1

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()